In [75]:
import os

inputs = []
label = []

classes = []

dirts = 'ws_data'

for dirt in os.listdir(dirts):
    files_path = os.path.join(dirts, dirt)
    for file in os.listdir(files_path):
        text = ''
        file_path = os.path.join(files_path, file)
        with open(file_path, 'r', encoding='utf-8') as f:
            for l in f:
                text = text + l

        if text.strip() != '':
            inputs.append(text)
            label.append(dirt)

### Preprocessing

In [86]:
import pandas as pd

docs = pd.DataFrame()
docs['text'] = inputs
docs['class'] = label
classes = docs['class'].unique().tolist()

classes

['Health', 'Art & Science', 'Goverment & Politics', 'Finance']

### Extracting Features

In [77]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

stop_words = text.ENGLISH_STOP_WORDS.union()
count_vect = CountVectorizer(token_pattern="[^\W\d_]+")
train_data_features = count_vect.fit_transform(docs['text'])

def preprocess_file(file_path):
    text = ''
    with open(file_path, 'r', errors='ignore') as f:
        for l in f:
            text = l + ' '
    return count_vect.transform([text])

def preprocess(input):
    return count_vect.transform(input)

### Using Multinomial Naive Bayes Model from sklearn

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

docs['class'] = docs['class'].replace(docs['class'].unique().tolist(), range(0, len(docs['class'].unique())))
X = train_data_features.toarray()
Y = docs['class'].tolist()

x, _x, y, _y = train_test_split(X, Y, test_size=0.1, random_state=42)

clf = MultinomialNB()
_ = clf.fit(x, y)

### Testing the Model

In [79]:
from sklearn.metrics import confusion_matrix
import numpy as np

yhat = clf.predict(_x)
matrix = confusion_matrix(_y, yhat)
accuracy = (matrix.diagonal() / matrix.sum(axis=1)) * 100

acc_table = pd.DataFrame()
acc_table['category'] = classes
acc_table['accuracy (%)'] = accuracy.round(1)

print(acc_table, '\n\nTotal Accuracy: ', (np.mean(yhat == _y)).round(1) * 100, '%')

               category  accuracy (%)
0                Health          72.7
1         Art & Science          98.2
2  Goverment & Politics          70.7
3               Finance          77.3 

Total Accuracy:  80.0 %


### Manual Test

In [1]:
docs_text = ['']

p_docs_text = preprocess(docs_text)
index = clf.predict(p_docs_text)[0]
print(classes[index])

NameError: name 'preprocess' is not defined